# Quick look at the data

In [3]:
import pandas as pd

In [5]:
col_names = ["Numero de cuenta", "Oficina", "Divisa", "Fecha operacion", "Fecha valor", 
               "Ingreso (+)", "Gasto (-)", "Saldo (+)", "Saldo (-)", "Concepto comun",
               "Concepto propio", "Referencia 1", "Referencia 2", "Concepto complementario 1",
               "Concepto complementario 2"]

finance = pd.read_csv("finance_2l_utf.csv", header = None, index_col=False, names = col_names, sep = ';')

In [6]:
finance.head(5)

,Numero de cuenta,Oficina,Divisa,Fecha operacion,Fecha valor,Ingreso (+),Gasto (-),Saldo (+),Saldo (-),Concepto comun,Concepto propio,Referencia 1,Referencia 2,Concepto complementario 1,Concepto complementario 2
0,2100 0889 40 0100978503,9736,EUR,17/04/2019,17/04/2019,NaN,"1,73","126,98",NaN,12,40,0,"4,59986E+15",,BON AREA
1,2100 0889 40 0100978503,9792,EUR,17/04/2019,17/04/2019,8,NaN,"128,71",NaN,2,2,0,ORIGEN: 00810900,00810900-ADRIA FERRER ABAD,TRANSF. A SU FAVOR
2,2100 0889 40 0100978503,9792,EUR,17/04/2019,17/04/2019,9,NaN,"120,71",NaN,2,2,0,ORIGEN: 00810053,00810053-PAU SAMPIETRO FABREGAS,TRANSF. A SU FAVOR
3,2100 0889 40 0100978503,9736,EUR,16/04/2019,16/04/2019,NaN,25,"111,71",NaN,12,40,0,"4,59986E+15",,PECUNIA CARDS EDE
4,2100 0889 40 0100978503,569,EUR,16/04/2019,16/04/2019,NaN,"35,08","136,71",NaN,3,1,0,20000005147,A66098435000,AIGUES DE BARC


In [8]:
finance.tail(1)

,Numero de cuenta,Oficina,Divisa,Fecha operacion,Fecha valor,Ingreso (+),Gasto (-),Saldo (+),Saldo (-),Concepto comun,Concepto propio,Referencia 1,Referencia 2,Concepto complementario 1,Concepto complementario 2
1224,2100 0889 40 0100978503,9736,EUR,20/04/2015,20/04/2015,NaN,"7,55","64,02",NaN,12,40,0,"4,59986E+15",,ALIBRI LLIBRERIA


#### Last transaction in data is on 20/04/2015

In [327]:
finance.info() # Found some null values in ["ingreso", "gasto"], ["saldo(+)", "saldo(+)"] -> seem to be complementary features

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225 entries, 0 to 1224
Data columns (total 15 columns):
Numero de cuenta             1225 non-null object
Oficina                      1225 non-null int64
Divisa                       1225 non-null object
Fecha operacion              1225 non-null object
Fecha valor                  1225 non-null object
Ingreso (+)                  147 non-null object
Gasto (-)                    1078 non-null object
Saldo (+)                    1224 non-null object
Saldo (-)                    1 non-null object
Concepto comun               1225 non-null int64
Concepto propio              1225 non-null int64
Referencia 1                 1225 non-null object
Referencia 2                 1225 non-null object
Concepto complementario 1    1225 non-null object
Concepto complementario 2    1225 non-null object
dtypes: int64(3), object(12)
memory usage: 143.6+ KB


# Drop columns that are not necessary for this analysis

#### Critical data columns that won't be dropped are the following:

#### "Fecha operacion", "Ingreso (+)", "Gasto (-)", "Saldo (+)", "Saldo (-)", "Concepto complementario 1", "Concepto complementario 2"

In [328]:
finance["Numero de cuenta"].value_counts().count() # Account number is always the same because the data is from one single account -> UNNECESSARY

1

#### From now on same account number is assumed -> DROP

In [329]:
finance["Oficina"].value_counts().count()

28

#### There are 28 different offices -> Maybe this feature is useful in the future

In [330]:
finance["Divisa"].value_counts() # Account is european (shared currency EUR) and clearly hasn't had any other currency -> UNNECESSARY

EUR    1225
Name: Divisa, dtype: int64

#### From now on single EUR currency is assumed -> DROP

In [331]:
finance[finance["Fecha operacion"] != finance['Fecha valor']].count()[0] # There are 59

59

#### Some operations were finished pressumably days after the operation request -> Maybe 'Fecha valor' is useful in the future?

#### After having a look at tail of data it is safe to say that transactions follow "Fecha operacion" -> DROP "Fecha valor" 

In [9]:
finance["Concepto comun"].value_counts().head(3)

12    936
2     145
11     56
Name: Concepto comun, dtype: int64

In [10]:
finance["Concepto propio"].value_counts().head(3)

40    1072
67      49
2       24
Name: Concepto propio, dtype: int64

#### Columns "Concepto comun" and "Concepto propio" are not clear what they mean -> UNCLEAR MEANING -> DROP

In [334]:
finance["Referencia 1"].value_counts()

0              1221
7,90013E+11       2
81809083119       1
81802330129       1
Name: Referencia 1, dtype: int64

In [11]:
finance["Referencia 2"].value_counts().head(3)

4,59986E+15         1069
                      63
20000005147           20
Name: Referencia 2, dtype: int64

#### "Referencia 1" and "Referencia 2" give little to no information -> UNCLEAR MEANING -> DROP

In [12]:
finance["Concepto complementario 1"].value_counts().head(3)

                                          1136
A66098435000                                20
01823298-MIREIA GOMEZ GORDO                 17
Name: Concepto complementario 1, dtype: int64

In [13]:
finance["Concepto complementario 2"].value_counts().head(3)

REINT.CAJERO                              57
LIDL C PUJADES                            54
CAFETERIA DEL MOL                         53
Name: Concepto complementario 2, dtype: int64

#### "Concepto complementario 1" is from where a incoming transaction is coming and "Concepto complementario 2" is to where -> Useful

In [338]:
finance['Saldo (-)'].value_counts()

-11,84    1
Name: Saldo (-), dtype: int64

#### There is only one row with negative balance, probably can be dropped after merging

In [339]:
to_drop = ["Numero de cuenta", "Divisa", "Concepto comun", "Concepto propio", "Referencia 1", "Referencia 2", 'Fecha valor']

finance.drop(to_drop, inplace=True, axis=1)

In [340]:
finance.head() # Result of dropping

,Oficina,Fecha operacion,Ingreso (+),Gasto (-),Saldo (+),Saldo (-),Concepto complementario 1,Concepto complementario 2
0,9736,17/04/2019,NaN,"1,73","126,98",NaN,,BON AREA
1,9792,17/04/2019,8,NaN,"128,71",NaN,00810900-ADRIA FERRER ABAD,TRANSF. A SU FAVOR
2,9792,17/04/2019,9,NaN,"120,71",NaN,00810053-PAU SAMPIETRO FABREGAS,TRANSF. A SU FAVOR
3,9736,16/04/2019,NaN,25,"111,71",NaN,,PECUNIA CARDS EDE
4,569,16/04/2019,NaN,"35,08","136,71",NaN,A66098435000,AIGUES DE BARC


# Convert types

#### Convert "Fecha operacion" and "Fecha valor" to datetime format

In [341]:
finance["Fecha operacion"] = pd.to_datetime(finance["Fecha operacion"], format='%d/%m/%Y')

#### Convert "Ingreso (+)", "Gasto (-)", "Saldo (+)" and "Saldo (-)" to numeric value

In [342]:
def banking_to_numeric(df, column):
    # Replace dots with blankspace (1.000 -> 1000)
    df[column] = df[column].str.replace('.', '', regex=True)
    
    # Replace commas for dots (1,5 -> 1.5)
    df[column] = df[column].str.replace(',', '.', regex=True)
    
    # Convert to numeric
    df[column] = pd.to_numeric(df[column], errors='raise')

In [343]:
to_num_col = ["Ingreso (+)", "Gasto (-)", "Saldo (+)", "Saldo (-)"]

for col in to_num_col:
    banking_to_numeric(finance, col)

In [344]:
finance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225 entries, 0 to 1224
Data columns (total 8 columns):
Oficina                      1225 non-null int64
Fecha operacion              1225 non-null datetime64[ns]
Ingreso (+)                  147 non-null float64
Gasto (-)                    1078 non-null float64
Saldo (+)                    1224 non-null float64
Saldo (-)                    1 non-null float64
Concepto complementario 1    1225 non-null object
Concepto complementario 2    1225 non-null object
dtypes: datetime64[ns](1), float64(4), int64(1), object(2)
memory usage: 76.6+ KB


# Deal with NaN values

#### Knowing that NaN values are complementary attributes (ex: income vs expense) it is safe to fill all with their complementary in another column.

In [345]:
finance["Ingreso y Gasto"] = finance["Ingreso (+)"].fillna(finance["Gasto (-)"] * -1) # It is useful to leave expenses and income separated cols

In [346]:
finance['Saldo'] = finance["Saldo (+)"].fillna(finance["Saldo (-)"])

In [347]:
finance.drop(["Saldo (-)", "Saldo (+)"], inplace=True, axis=1) # These columns have been merged into another so they can be droped

# Rename columns to more useful ones

In [348]:
new_names =  {'Concepto complementario 1': 'Concepto origen',
              'Concepto complementario 2': 'Concepto destino',
              'Saldo (+)': 'Saldo'}

finance.rename(columns=new_names, inplace=True)

In [349]:
finance.columns

Index(['Oficina', 'Fecha operacion', 'Ingreso (+)', 'Gasto (-)',
       'Concepto origen', 'Concepto destino', 'Ingreso y Gasto', 'Saldo'],
      dtype='object')

# Export clean data

In [16]:
finance.head()

,Numero de cuenta,Oficina,Divisa,Fecha operacion,Fecha valor,Ingreso (+),Gasto (-),Saldo (+),Saldo (-),Concepto comun,Concepto propio,Referencia 1,Referencia 2,Concepto complementario 1,Concepto complementario 2
0,2100 0889 40 0100978503,9736,EUR,17/04/2019,17/04/2019,NaN,"1,73","126,98",NaN,12,40,0,"4,59986E+15",,BON AREA
1,2100 0889 40 0100978503,9792,EUR,17/04/2019,17/04/2019,8,NaN,"128,71",NaN,2,2,0,ORIGEN: 00810900,00810900-ADRIA FERRER ABAD,TRANSF. A SU FAVOR
2,2100 0889 40 0100978503,9792,EUR,17/04/2019,17/04/2019,9,NaN,"120,71",NaN,2,2,0,ORIGEN: 00810053,00810053-PAU SAMPIETRO FABREGAS,TRANSF. A SU FAVOR
3,2100 0889 40 0100978503,9736,EUR,16/04/2019,16/04/2019,NaN,25,"111,71",NaN,12,40,0,"4,59986E+15",,PECUNIA CARDS EDE
4,2100 0889 40 0100978503,569,EUR,16/04/2019,16/04/2019,NaN,"35,08","136,71",NaN,3,1,0,20000005147,A66098435000,AIGUES DE BARC


In [14]:
finance.tail(1)

,Numero de cuenta,Oficina,Divisa,Fecha operacion,Fecha valor,Ingreso (+),Gasto (-),Saldo (+),Saldo (-),Concepto comun,Concepto propio,Referencia 1,Referencia 2,Concepto complementario 1,Concepto complementario 2
1224,2100 0889 40 0100978503,9736,EUR,20/04/2015,20/04/2015,NaN,"7,55","64,02",NaN,12,40,0,"4,59986E+15",,ALIBRI LLIBRERIA


In [352]:
finance.to_csv('finance_clean.csv', index=False)